<a href="https://colab.research.google.com/github/meliy-meyada/Tensorflow-Deep-Learning-and-Artificial-Intelligence/blob/main/TF2_0Mirrored_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x #Colab only.
except Exception:
  pass

import tensorflow as tf

print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.8.2


In [2]:
# More additional imports

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPool2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [3]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)

170508288/170498071 [==============================] - 3s 0us/step
x_train.shape:  (50000, 32, 32, 3)
y_train.shape:  (50000,)


In [5]:
# number of classes
K = len(set(y_train))
print("number of classes: ", K)

number of classes:  10


In [6]:
# Build the model using the functional API
def create_model():
  i = Input(shape=x_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model

In [7]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.experimental.CentralStorageStrategy()

In [8]:
print(f"Number of devices: {strategy.num_replicas_in_sync}")

Number of devices: 1


In [9]:
with strategy.scope():
  model = create_model()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [10]:
# Fit
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=15)

Epoch 1/15
391/391 [==============================] - 23s 20ms/step - loss: 1.4722 - accuracy: 0.5052 - val_loss: 2.0573 - val_accuracy: 0.3222
Epoch 2/15
391/391 [==============================] - 6s 15ms/step - loss: 0.8636 - accuracy: 0.6936 - val_loss: 0.8161 - val_accuracy: 0.7129
Epoch 3/15
391/391 [==============================] - 6s 15ms/step - loss: 0.6738 - accuracy: 0.7648 - val_loss: 0.7135 - val_accuracy: 0.7551
Epoch 4/15
391/391 [==============================] - 6s 15ms/step - loss: 0.5467 - accuracy: 0.8085 - val_loss: 0.7161 - val_accuracy: 0.7616
Epoch 5/15
391/391 [==============================] - 6s 15ms/step - loss: 0.4631 - accuracy: 0.8374 - val_loss: 0.8125 - val_accuracy: 0.7356
Epoch 6/15
391/391 [==============================] - 6s 15ms/step - loss: 0.3764 - accuracy: 0.8669 - val_loss: 0.6123 - val_accuracy: 0.8025
Epoch 7/15
391/391 [==============================] - 6s 15ms/step - loss: 0.3050 - accuracy: 0.8918 - val_loss: 0.7434 - val_accuracy: 0.774

In [11]:
50000/391

127.8772378516624

In [12]:
10000/79

126.58227848101266

In [13]:
model2 = create_model()

model2.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=5)

Epoch 1/5
391/391 [==============================] - 6s 13ms/step - loss: 1.4175 - accuracy: 0.5240 - val_loss: 2.2556 - val_accuracy: 0.2866
Epoch 2/5
391/391 [==============================] - 5s 12ms/step - loss: 0.8598 - accuracy: 0.6973 - val_loss: 0.8727 - val_accuracy: 0.6939
Epoch 3/5
391/391 [==============================] - 5s 12ms/step - loss: 0.6630 - accuracy: 0.7664 - val_loss: 0.7318 - val_accuracy: 0.7496
Epoch 4/5
391/391 [==============================] - 5s 12ms/step - loss: 0.5432 - accuracy: 0.8104 - val_loss: 0.7873 - val_accuracy: 0.7310
Epoch 5/5
391/391 [==============================] - 5s 12ms/step - loss: 0.4555 - accuracy: 0.8410 - val_loss: 0.6120 - val_accuracy: 0.7929
